In [1]:
import keras
from keras.preprocessing import image
from keras.engine import Layer
from keras.layers import Conv2D, Conv3D, UpSampling2D, InputLayer, Conv2DTranspose, Input, Reshape, merge, concatenate
from keras.layers import Activation, Dense, Dropout, Flatten
from keras.layers.normalization import BatchNormalization
from keras.callbacks import TensorBoard
from keras.models import Sequential, Model, load_model
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img
from skimage.color import rgb2lab, lab2rgb, rgb2gray, gray2rgb
from skimage.transform import resize
from skimage.io import imsave
from time import time
import numpy as np
import os
import random
import tensorflow as tf
from PIL import Image, ImageFile

In [2]:
model = load_model('Downloads/colorizationModelVGG.hdf5')

In [3]:
vggmodel = keras.applications.vgg16.VGG16()
newmodel = Sequential() 
num = 0
for i, layer in enumerate(vggmodel.layers):
    if i<19:
      newmodel.add(layer)
newmodel.summary()
for layer in newmodel.layers:
  layer.trainable=False

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
block1_conv1 (Conv2D)        (None, 224, 224, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 224, 224, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 112, 112, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 112, 112, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 112, 112, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 56, 56, 128)       0         
_________________________________________________________________
block3_conv1 (Conv2D)        (None, 56, 56, 256)       2

In [6]:
from numpy import asarray
L_part = []
AB = []
for image_name in os.listdir("Downloads/test/"):
    test_image = Image.open("Downloads/test/"+image_name)
    test_image = test_image.resize((224, 224))
    test_image = asarray(test_image)
    print(image_name)
    try:
        lab_test = rgb2lab(test_image)
    except:
        test_gray = gray2rgb(test_image)
        lab_test = rgb2lab(test_gray)
    lab_test = lab_test.astype('float32')
    lab_test[:,:,0] /= 100
    L_part.append(lab_test[:,:,0])
    L = gray2rgb(lab_test[:,:,0])
    L = gray2rgb(lab_test[:,:,0])
    L = L.reshape((1,224,224,3))
    vggpred = newmodel.predict(L)
    ab = model.predict(vggpred)
    ab *= 127
    AB.append(ab)
print(len(AB))

2016-06-27-black-white-photo-06.jpg
22art2-1.jpg
dia mirza.jpg
elephant.jpg
images (1).jfif
images (2).jfif
images.jfif
lion.jpg
main-qimg-52365feb99da5b9bce6ae216c18e6660.jfif
matt_miller_embed-710x1066.jpg
missing-you-black-white-16655.jpg
p-364-jackanstey-03283-jj.jpg
photo-1501059520966-c59141726d6c.jfif
road.jpg
xovk0byrmkcuvn0vjnyoxg.jpg
15


In [8]:
for i in range(len(AB)):
    cur = np.zeros((224, 224, 3))
    cur[:,:,0] = L_part[i]*100
    cur[:,:,1:] = AB[i]
    rgb = lab2rgb(cur)
    rgb *= 255
    rgb = rgb.astype('uint8')
    imsave("Downloads/results3/result_vgg"+str(i)+".png", rgb)